In [ ]:
from langchain_openai import ChatOpenAI
import pandas as pd
from dotenv import load_dotenv
from datasets import load_dataset, Dataset


from langchain_openai.chat_models import ChatOpenAI
from langchain_core.tools import Tool
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_openai import ChatOpenAI

from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

load_dotenv()

In [ ]:

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")

In [ ]:

# Plan-and-execute response 
def generate_response(prompt_input):
    model = ChatOpenAI(model= "gpt-4o-mini", temperature=0.7)
    
    planner = load_chat_planner(model)
    tools = []
    
    executor = load_agent_executor(model, tools, verbose=True)
    
    agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)
    response = agent.invoke({"input": prompt_input})
    return response["output"]

In [ ]:

# Generate answers for a given model 
ds_for_parse = amnesty_qa["eval"].to_pandas()
results = []
prompts = []
for index, row in ds_for_parse.iterrows(): 
    context = row["contexts"]
    question = row["question"]
    prompt_input = f"""Answer the question: {question}
    Use the provided context to answer the question. The relevant contexts extracted from relevant documents are listed below:
    {context}"""
    prompts.append(prompt_input)
    result = generate_response(prompt_input)
    results.append(result)

In [ ]:
my_dataset = amnesty_qa["eval"].to_pandas()
my_dataset["answer"] = pd.Series(results)
my_ds_for_eval = Dataset.from_pandas(my_dataset)

In [ ]:

result = evaluate(
    my_ds_for_eval,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)